In [ ]:
#User Registration
pip install opencv-python face_recognition Flask faiss-cpu


In [13]:
#Option 1

import os
import shutil
import cv2
import face_recognition
import numpy as np
import faiss
from datetime import datetime

# Initialize FAISS index (FlatL2 for simplicity)
dimension = 128  # Face encoding dimension
index = faiss.IndexFlatL2(dimension)

# Database to store user info
user_db = {}

def register_user_with_webcam(emp_id, emp_name):
    # Get the current system date in YYYYMMDD format
    system_date = datetime.now().strftime("%Y%m%d")

    # Define the base directory path
    base_directory = os.path.join("data", "registered", system_date)

    # Create the directory if it doesn't exist
    if not os.path.exists(base_directory):
        os.makedirs(base_directory)
    else:
        # If it exists, clear its contents using shutil.rmtree and then recreate it
        shutil.rmtree(base_directory)
        os.makedirs(base_directory)

    # User-specific folder within the base directory
    user_folder = os.path.join(base_directory, emp_id)
    if not os.path.exists(user_folder):
        os.makedirs(user_folder)

    # Capture image from the webcam
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Register")

    print("Press SPACE to capture the image and ESC to exit.")
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break
        cv2.imshow("Register", frame)

        k = cv2.waitKey(1)
        if k % 256 == 27:  # ESC pressed
            print("Escape hit, closing...")
            break
        elif k % 256 == 32:  # SPACE pressed
            img_name = os.path.join(user_folder, f"{emp_id}.jpg")
            cv2.imwrite(img_name, frame)
            print(f"{img_name} saved!")
            break

    cam.release()
    cv2.destroyAllWindows()

    # Load the image and convert it to RGB
    image = cv2.imread(img_name)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect face and encode
    face_locations = face_recognition.face_locations(rgb_image)
    face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

    if len(face_encodings) > 0:
        face_encoding = face_encodings[0]
        # Store in FAISS
        index.add(np.array([face_encoding]))
        # Map emp_id to index and store user info
        user_db[emp_id] = {
            "index": index.ntotal - 1,
            "name": emp_name,
            "emp_id": emp_id,
            "registration_date": system_date,
            "image_path": img_name
        }
        print(f"User {emp_id} - {emp_name} registered successfully in {base_directory}.")
    else:
        print("No face found in the image.")

# Example usage
emp_id = input("Enter your Employee ID: ")
emp_name = input("Enter your Name: ")

register_user_with_webcam(emp_id, emp_name)


Enter your Employee ID:  1356311
Enter your Name:  Latha


Press SPACE to capture the image and ESC to exit.
data\registered\20240703\1356311\1356311.jpg saved!
User 1356311 - Latha registered successfully in data\registered\20240703.


In [14]:
#User Authentication

import os
import shutil
import cv2
import face_recognition
import numpy as np
import faiss
from datetime import datetime

# Initialize FAISS index (FlatL2 for simplicity)
dimension = 128  # Face encoding dimension
index = faiss.IndexFlatL2(dimension)

# Database to store user info
user_db = {}

def register_user_with_webcam(emp_id, emp_name):
    # Get the current system date in YYYYMMDD format
    system_date = datetime.now().strftime("%Y%m%d")

    # Define the base directory path
    base_directory = os.path.join("data", "registered", system_date)

    # Create the directory if it doesn't exist
    if not os.path.exists(base_directory):
        os.makedirs(base_directory)
    else:
        # If it exists, clear its contents using shutil.rmtree and then recreate it
        shutil.rmtree(base_directory)
        os.makedirs(base_directory)

    # User-specific folder within the base directory
    user_folder = os.path.join(base_directory, emp_id)
    if not os.path.exists(user_folder):
        os.makedirs(user_folder)

    # Capture image from the webcam
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Register")

    print("Press SPACE to capture the image and ESC to exit.")
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break
        cv2.imshow("Register", frame)

        k = cv2.waitKey(1)
        if k % 256 == 27:  # ESC pressed
            print("Escape hit, closing...")
            break
        elif k % 256 == 32:  # SPACE pressed
            img_name = os.path.join(user_folder, f"{emp_id}.jpg")
            cv2.imwrite(img_name, frame)
            print(f"{img_name} saved!")
            break

    cam.release()
    cv2.destroyAllWindows()

    # Load the image and convert it to RGB
    image = cv2.imread(img_name)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect face and encode
    face_locations = face_recognition.face_locations(rgb_image)
    face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

    if len(face_encodings) > 0:
        face_encoding = face_encodings[0]
        # Store in FAISS
        index.add(np.array([face_encoding]))
        # Map emp_id to index and store user info
        user_db[emp_id] = {
            "index": index.ntotal - 1,
            "name": emp_name,
            "emp_id": emp_id,
            "registration_date": system_date,
            "image_path": img_name
        }
        print(f"User {emp_id} - {emp_name} registered successfully in {base_directory}.")
    else:
        print("No face found in the image.")

def verify_user_with_webcam(emp_id):
    if emp_id not in user_db:
        print(f"No user found with Employee ID: {emp_id}")
        return False

    # Capture image from the webcam
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Verify")

    print("Press SPACE to capture the image and ESC to exit.")
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break
        cv2.imshow("Verify", frame)

        k = cv2.waitKey(1)
        if k % 256 == 27:  # ESC pressed
            print("Escape hit, closing...")
            break
        elif k % 256 == 32:  # SPACE pressed
            img_name = f"temp_{emp_id}.jpg"
            cv2.imwrite(img_name, frame)
            print(f"{img_name} saved!")
            break

    cam.release()
    cv2.destroyAllWindows()

    # Load the image and convert it to RGB
    image = cv2.imread(img_name)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect face and encode
    face_locations = face_recognition.face_locations(rgb_image)
    face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

    if len(face_encodings) > 0:
        live_encoding = face_encodings[0]
        # Retrieve stored encoding
        stored_index = user_db[emp_id]["index"]
        stored_encoding = index.reconstruct(stored_index)

        # Compare encodings
        distances, _ = index.search(np.array([live_encoding]), k=1)
        threshold = 0.6  # This is an example threshold for similarity
        if distances[0][0] < threshold:
            print("Face match found")
            return True
        else:
            print("No face match found")
            return False
    else:
        print("No face found in the image.")
        return False

# Example usage
emp_id = input("Enter your Employee ID: ")
emp_name = input("Enter your Name: ")

register_user_with_webcam(emp_id, emp_name)

# Verification
emp_id = input("Enter your Employee ID for verification: ")
verify_user_with_webcam(emp_id)


Enter your Employee ID:  1356311
Enter your Name:  Latha


Press SPACE to capture the image and ESC to exit.
data\registered\20240703\1356311\1356311.jpg saved!
User 1356311 - Latha registered successfully in data\registered\20240703.


Enter your Employee ID for verification:  1356311


Press SPACE to capture the image and ESC to exit.
temp_1356311.jpg saved!
Face match found


True